In [1]:
print()

In [14]:
from langgraph.graph import StateGraph, add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_groq import ChatGroq
from typing import TypedDict, Annotated, List

# Step 1: Define your memory state
class ChatState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# Step 2: Initialize Groq's model (Mixtral)
llm = ChatGroq(model="llama-3.1-8b-instant")  # or "llama3-70b-8192", etc.

# Step 3: Define the LLM node function
def qa_node(state: ChatState) -> ChatState:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}  # merged with previous via `add_messages`

# Step 4: Use a checkpointer (in-memory for now)
memory = MemorySaver()

# Step 5: Build the LangGraph
builder = StateGraph(ChatState)
builder.add_node("qa", qa_node)
builder.set_entry_point("qa")
builder.set_finish_point("qa")

graph = builder.compile(checkpointer=memory)


In [15]:
# Define a unique session ID (can be user_id, etc.)
session_id = "groq-chat-session-1"
config = {"configurable": {"thread_id": session_id}}

# Turn 1
response1 = graph.invoke({
    "messages": [HumanMessage(content="what is the capital of india?")]
}, config=config)

print("Bot:", response1["messages"][-1].content)

# Turn 2 — continues with memory
response2 = graph.invoke({
    "messages": [HumanMessage(content="where tajmal is there?")]
}, config=config)

print("Bot:", response2["messages"][-1].content)


Bot: The capital of India is New Delhi.
Bot: You're probably referring to Taj Mahal. The Taj Mahal is located in Agra, a city in the state of Uttar Pradesh, India. It is one of the Seven Wonders of the World and a UNESCO World Heritage Site.


In [17]:
response3 = graph.invoke({
    "messages": [HumanMessage(content="what is my previoud question?")]
}, config=config)

print("Bot:", response3["messages"][-1].content)


Bot: Your previous question was "where tajmal is there?"


In [13]:
response3 = graph.invoke({
    "messages": [HumanMessage(content="what is my previous question?")]
}, config=config)

print("Bot:", response3["messages"][-1].content)


Bot: Your previous question was "Can you explain it simply?"


In [10]:
graph.get_state(config)


StateSnapshot(values={'messages': [HumanMessage(content='What is quantum physics?', additional_kwargs={}, response_metadata={}, id='59d4a572-8c94-43d8-a98d-c344a82cbed8'), AIMessage(content="Quantum physics, also known as quantum mechanics, is a branch of physics that studies the behavior of matter and energy at an atomic and subatomic level. At these scales, the classical laws of physics no longer apply, and strange, seemingly random phenomena occur.\n\nThe core principles of quantum physics include:\n\n1. **Wave-particle duality**: Quantum objects, such as electrons and photons, can exhibit both wave-like and particle-like behavior.\n2. **Uncertainty principle**: It's impossible to know certain properties of a quantum object, like its position and momentum, simultaneously with infinite precision.\n3. **Superposition**: Quantum objects can exist in multiple states simultaneously, like spinning both clockwise and counterclockwise at the same time.\n4. **Entanglement**: Quantum objects 

In [11]:
list(graph.get_state_history(config))


[StateSnapshot(values={'messages': [HumanMessage(content='What is quantum physics?', additional_kwargs={}, response_metadata={}, id='59d4a572-8c94-43d8-a98d-c344a82cbed8'), AIMessage(content="Quantum physics, also known as quantum mechanics, is a branch of physics that studies the behavior of matter and energy at an atomic and subatomic level. At these scales, the classical laws of physics no longer apply, and strange, seemingly random phenomena occur.\n\nThe core principles of quantum physics include:\n\n1. **Wave-particle duality**: Quantum objects, such as electrons and photons, can exhibit both wave-like and particle-like behavior.\n2. **Uncertainty principle**: It's impossible to know certain properties of a quantum object, like its position and momentum, simultaneously with infinite precision.\n3. **Superposition**: Quantum objects can exist in multiple states simultaneously, like spinning both clockwise and counterclockwise at the same time.\n4. **Entanglement**: Quantum objects

In [12]:
messages = memory.get_messages("groq-chat-session-1")
print(messages)


AttributeError: 'InMemorySaver' object has no attribute 'get_messages'

In [31]:
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState

# Step 1: Define your memory state
class ChatState(MessagesState):
    result:str

# Step 2: Initialize Groq's model (Mixtral)
llm = ChatGroq(model="llama-3.1-8b-instant")  # or "llama3-70b-8192", etc.

# Step 3: Define the LLM node function
def qa_node(state: ChatState) -> ChatState:
    response = llm.invoke(state["messages"][-1].content)
    return {"messages": [response],"result":response.content}  # merged with previous via `add_messages`

# Step 4: Use a checkpointer (in-memory for now)
memory = MemorySaver()

# Step 5: Build the LangGraph
builder = StateGraph(ChatState)
builder.add_node("qa", qa_node)
builder.set_entry_point("qa")
builder.set_finish_point("qa")

graph = builder.compile(checkpointer=memory)


In [32]:
# Define a unique session ID (can be user_id, etc.)
session_id = "1"
config = {"configurable": {"thread_id": session_id}}

# Turn 1
response11 = graph.invoke({
    "messages": [HumanMessage(content="what is the capital of india?")]
}, config=config)

print("Bot:", response11["messages"][-1].content)


Bot: The capital of India is New Delhi.


In [33]:
response11 = graph.invoke({
    "messages": [HumanMessage(content="what is my previous question")]
}, config=config)

print("Bot:", response11["messages"][-1].content)

Bot: This is the beginning of our conversation, therefore I have no record of a previous question from you. How can I assist you today?


In [34]:
response11["result"]

'This is the beginning of our conversation, therefore I have no record of a previous question from you. How can I assist you today?'